# Metodo Simple Maximizacion 

In [ ]:
def fill_table(n, m):
    # Inicializar la tabla con ceros
    row = m + 1
    col = m + n + 1
    table = [[0 for _ in range(col)] for _ in range(row)]

    # Pedir y llenar las restricciones
    for i in range(row - 1):
        constraint_coefficients = list(map(float, input(
            f"Ingrese los coeficientes de la restricción {i + 1} (coeficientes separados por espacio):").split()))
        for j in range(n):
            table[i][j] = constraint_coefficients[j]
        # Coeficiente de la variable de holgura para la restricción i
        table[i][n + i] = 1  # Variable de holgura

        # Término independiente
        table[i][col - 1] = constraint_coefficients[n]

    # Pedir la función objetivo
    objective_function = list(map(float, input(
        "Ingrese los coeficientes de la función objetivo (coeficientes separados por espacio):").split()))

    # Llenar la última fila de la tabla con la función objetivo
    for j in range(n):
        table[row - 1][j] = -1*objective_function[j]
    
    return table 

In [ ]:
def get_optimos(n, row, col, table):
    count_ones = 0
    indice = 0
    x_opt = [0] * n
    for j in range(n): # x_j
        for i in range(row):
            # Si no es 0 o 1 se pasa a la siguiente columna
            if table[i][j]!=0 or table[i][j]!=1: break
            
            # Valor valido entonces contamos 1's y guardamos indice de las filas
            if table[i][j]==1: 
                indice = i
                count_ones += 1 
        
        # Si no hallo ningun 1, o mas de un 1's
        if count_ones != 1: continue
        
        # Acabo de recorred las filas de la columna actual
        x_opt[j] = table[col-1][indice]
    
    return x_opt
            

In [ ]:
# Metodo para devolver los indices del pivote optimo
def index_pivot(n, row, col, table):
    index = (-1, -1)
    # --- Columna
    # Buscamos el mayor negativo entre x1, x2, x3, xn
    last = row - 1
    max_value = 0
    for j in range(n):
        if table[last][j] < max_value:
            max_value = table[last][j]
            index = (last, j)
    # ---- Fila
    # El menor número positivo al dividir (biq/aiq)
    j_b = col - 1
    j_pivot = index[1]
    min_value = float('inf')
    for i in range(row - 1):
        if table[i][j_pivot] > 0:
            r = table[i][j_b] / table[i][j_pivot]
            if r < min_value:
                min_value = r
                index = (i, j_pivot)
    return index


In [ ]:
# Metodo para saber si aun son negativos en la ultima fila
def has_negatives(n, row, table):
    last = row - 1
    for j in range(n):
        if table[last][j] < 0:
            return True
    return False

In [ ]:
def print_table(table, n, m):
    headers = []
    # Encabezados para las variables x1, x2, ..., xn
    for i in range(n):
        headers.append(f'x{i + 1}')
    # Encabezados para las variables de holgura s1, s2, ..., sm
    for i in range(m):
        headers.append(f's{i + 1}')
    # Encabezado para el resultado r
    headers.append('r')

    # Imprimir los encabezados
    print("\t".join(headers))

    # Imprimir la tabla
    for row in table:
        print("\t".join(map(lambda x: "{:.2f}".format(x), row)))


In [ ]:
# Realizar operaciones fila
def convert_table(row, col, pivot_row, pivot_column, table):
    ratio = table[pivot_row][pivot_column]
    for i in range(row):
        if i != pivot_row:
            factor = table[i][pivot_column] / ratio
            for j in range(col):
                table[i][j] -= factor * table[pivot_row][j]

In [ ]:
def simplex_method_maximize():
    n = int(input("Número de variables: "))
    m = int(input("Número de restricciones: "))
    row = m + 1
    col = m + n + 1
    table = fill_table(n, m)
    
    print("\nEntrando al bucle.....\n")
    
    while True:
        pivot_row, pivot_column = index_pivot(n, row, col, table)
        pivot_value = table[pivot_row][pivot_column]
        print(f"Nuestro pivote[{pivot_row}][{pivot_column}] = {pivot_value}")
        
        # Convertir el pivote a 1 y hacer ceros en la columna del pivote
        for j in range(col):
            table[pivot_row][j] /= pivot_value
        
        convert_table(row, col, pivot_row, pivot_column, table)
        
        print("\nActualizando tabla ..........\n")
        print_table(table, n, m)
        
        if not has_negatives(n, row, table):
            break

simplex_method_maximize()

# Metodo Simple Minimizacion

In [ ]:
def index_pivot_2(row, col, table, arr):
    last = row - 1
    min_diff = float('inf')
    pivot_column = -1
    for j in range(col - 1):
        diff = table[last][j]
        for i in range(row - 1):
            diff -= table[i][j] * arr[i]
        if diff < min_diff:
            min_diff = diff
            pivot_column = j

    if pivot_column == -1:
        return None  # No se encontró un pivote válido

    min_ratio = float('inf')
    pivot_row = -1
    j_b = col - 1
    for i in range(row - 1):
        if table[i][pivot_column] > 0:
            ratio = table[i][j_b] / table[i][pivot_column]
            if ratio < min_ratio:
                min_ratio = ratio
                pivot_row = i

    if pivot_row == -1:
        return None  # No se encontró una fila válida para el pivote

    # Cambiamos los valores de los coeficientes básicos
    arr[pivot_row] = table[last][pivot_column]
    return pivot_row, pivot_column

In [ ]:
def has_negatives_2(row, col, table, arr):
    last = row - 1
    for j in range(col - 1):
        diff = table[last][j]
        for i in range(row - 1):
            diff -= table[i][j] * arr[i]
        if diff < 0:
            return True
    return False

In [ ]:
M = float('inf')

def filltable2(n, m):
    row = m + 1
    col = n + 2 * m + 1

    # Inicializar la tabla con ceros
    table = [[0 for _ in range(col)] for _ in range(row)]

    # Pedir los coeficientes de la función objetivo
    print("Ingrese los coeficientes de la función objetivo (separados por espacio):")
    objective_coeffs = list(map(float, input().split()))
    
    # Llenar la última fila de la tabla con la función objetivo
    for j in range(n):
        table[row - 1][j] = objective_coeffs[j]
    
    j = n
    for i in range(m):
        table[row-1][j] = 0
        table[row-1][j+1] = M
        j += 2
    
    # Pedir y llenar las restricciones
    for i in range(0, row-1):
        print(f"Ingrese los coeficientes de la restricción {i+1} (separados por espacio):")
        constraint_coeffs = list(map(float, input().split()))
        for j in range(n):
            table[i][j] = constraint_coeffs[j]
        # Coeficiente de la variable de exceso para la restricción i
        table[i][n + i*2] = -1  # Variable de exceso S
        table[i][n + i*2 + 1] = 1       # Variable artificial A
        # Término independiente
        table[i][col - 1] = constraint_coeffs[n]

    return table

In [ ]:
def simplex_method_minimize():
    n = int(input("Numero de Variables: "))
    m = int(input("Numero de Restricciones: "))
    row = m + 1 
    col = n + 2 * m + 1

    # Array de los coeficientes Basicos
    arr = [M] * m

    # Rellenamos nuestra tabla
    table = filltable2(n, m)
    
    print_table(table,n,m)
    while True:
        # Identificamos nuestro pivote
        pivot_row, pivot_column = index_pivot_2(row, col, table, arr)
        pivot_value = table[pivot_row][pivot_column]
        print(f"\nNuestro pivote[{pivot_row}][{pivot_column}] = {pivot_value}")

        # Convertimos el pivote a 1 y hacemos ceros en la columna del pivote
        for j in range(col):
            table[pivot_row][j] /= pivot_value

        # Convertimos 0's en la columna del pivote mediante operaciones fila
        # OP : filaActual - razon*filaPivote
        convert_table(row-1, col, pivot_row, pivot_column, table)
        
        # Actualizamos la tabla
        print("\nActualizando tabla ..........\n")
        print_table(table,n,m)

        if not has_negatives_2(row, col, table, arr):
            break

# Ejecutamos el método
simplex_method_minimize()

# Metodo simple Maximizacion - Mixto


In [2]:
!pip install pulp

In [ ]:
from pulp import *
p1 = LpProblem("Metodo_Simplex_Mixto", LpMaximize)
x1 = LpVariable("X1")
x2 = LpVariable("X2")

In [ ]:
# Funcion objetivo
p1 += 7*x1 + 9*x2
# Restricciones
p1 += -x1 + 3*x2 <= 6
p1 += 7*x1 + x2 <= 35
p1 += x1 >=0
p1 += x2 >=0

In [ ]:
status = p1.solve()
print(value(x1))
print(value(x2))

# Metodo dual

# Metodo Branch and Bound

In [3]:
import pulp

def branch_and_bound(prob):
  """
  Resuelve un problema de programación lineal utilizando el algoritmo de ramificación y acotamiento.

  Parámetros:
    prob: Problema de programación lineal definido en Pulp.

  Devuelve:
    La solución óptima del problema.
  """

  # Resolver el problema como un modelo continuo

  prob_cont = prob.copy()
  prob_cont.solve()

  # Si la solución es entera, devolverla

  if prob_cont.status == pulp.LpStatusOptimal:
    return prob_cont.x

  # Generar los subproblemas ramificados

  for var in prob.variables:
    if not var.is_integer():
      prob_l = prob.copy()
      prob_l[var] = var.lb
      prob_u = prob.copy()
      prob_u[var] = var.ub
      yield from branch_and_bound(prob_l)
      yield from branch_and_bound(prob_u)



In [4]:
import pulp

def example():
  # Definir el problema de programación lineal

  prob = pulp.LpProblem("example", pulp.LpMaximize)

  x1 = pulp.LpVariable("x1", 0, None, pulp.LpInteger)
  x2 = pulp.LpVariable("x2", 0, None, pulp.LpInteger)

  prob += x1 + 5 * x2

  prob += 11 * x1 + 6 * x2 <= 66
  prob += 5 * x1 + 50 * x2 <= 225

  # Resolver el problema utilizando el algoritmo branch and bound

  sol = branch_and_bound(prob)

  # Imprimir la solución óptima

  print(sol)

if __name__ == "__main__":
  example()


<generator object branch_and_bound at 0x0000019669B387C0>
